In [1]:
# This is useful if you are working with a dev install of asf_search and experimenting with changes to the codebase
%load_ext autoreload
%autoreload 2

### Search for a geographic reference product from which to create an `asf_search.Stack`

This notebook uses a full Sentinel-1 scene as the reference product, but Sentinel-1 burst products are also supported.

In [2]:
import asf_search as asf

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
reference = results[0]
reference

### Create an `asf_search.Stack` object from the reference scene `asf_search.Products.S1Product.S1Product` object

Stack accepts ASFSearchOptions, which you can use to limit its temporal range.

This creates a `asf_search.Stack` based on the results for a given `asf.Stack` search.

`stack.full_stack` contains a complete graph, in which every scene is connected to every other scene.

In [3]:
args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-04-02'}
)

stack = asf.Stack(reference, opts=args)
stack.full_stack




[Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220323T225119_20220323T225146_042455_051017_6160),
 Pair(S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C, S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755

### At this point, `stack.full_stack` == `stack.subset_stack` because we haven't yet removed any pairs

Also note that `stack.remove_list` is empty, and `stack.subset_stack` is connected, which can be seen by the fact that `stack.connected_substacks` contains only a single item.

In [4]:
print(f'len(stack.full_stack): {len(stack.full_stack)}')
print(f'len(stack.subset_stack): {len(stack.subset_stack)}')
print(f'len(stack.remove_list): {len(stack.remove_list)}')
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')
print(f'stack.subset_stack == stack.full_stack: {stack.subset_stack == stack.full_stack}')

len(stack.full_stack): 21
len(stack.subset_stack): 21
len(stack.remove_list): 0
len(stack.connected_substacks): 1
stack.subset_stack == stack.full_stack: True


### Remove some Pairs from the stack

The `Pair`s have been added to `stack.remove_list` and removed from `stack.subset_stack`

In [5]:
stack.remove_pairs(stack.full_stack[1:11])

print(f'len(stack.full_stack): {len(stack.full_stack)}')
print(f'len(stack.subset_stack): {len(stack.subset_stack)}')
print(f'len(stack.remove_list): {len(stack.remove_list)}')
print(f'stack.subset_stack == stack.full_stack: {stack.subset_stack == stack.full_stack}')

len(stack.full_stack): 21
len(stack.subset_stack): 11
len(stack.remove_list): 10
stack.subset_stack == stack.full_stack: False


### Try removing a `Pair` that is not part of `stack.full_stack`

This will raise a `PairNotInFullStackWarning` indicating that the Pair cannot be removed since it is not part of the Stack

In [6]:
# Build a Pair from scenes that are not included in the existing Stack
results_1 = asf.product_search('S1A_IW_SLC__1SDV_20250903T225120_20250903T225147_060830_0792D3_868A-SLC')
results_2 = asf.product_search('S1A_IW_SLC__1SDV_20250822T225120_20250822T225147_060655_078BEA_2065-SLC')
my_pair = asf.Pair(results_1[0], results_2[0])

stack.remove_pairs([my_pair])

/Users/aflewandowski/Documents/asf_search/Discovery-asf_search/asf_search/Stack.py:83: PairNotInFullStackWarning: warning: Pair(S1A_IW_SLC__1SDV_20250903T225120_20250903T225147_060830_0792D3_868A, S1A_IW_SLC__1SDV_20250822T225120_20250822T225147_060655_078BEA_2065) is not in full_stack
  warnings.warn(PairNotInFullStackWarning(msg))


### Examine `stack.connected_substacks`

For illustrative purposes, the set of previously removed `Pair`s was intentionally selected to disconnect the `stack.subset_stack` into two connected `Stack` networks.

We can view these connected substacks by examing `stack.connected_substacks`, which contains a list of all connected substacks within `stack.subset_stack`

In [7]:
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')

for s in stack.connected_substacks:
    print(s)

len(stack.connected_substacks): 2
{Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C): Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C)}
{Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252): Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252), Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E): Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E), Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220311T

### Add one of the removed `Pair`s back to `stack.subset_stack`

Adding this `Pair` back once again fully-connects `stack.subset_stack`

In [8]:
stack.add_pairs([stack.remove_list[0]])
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')

len(stack.connected_substacks): 1


### You can also add `Pairs` that are not part of the original `stack.full_stack`

This is useful if you need to add custom `Pairs` that were not included during automated `Stack.full_stack` creation. We created such a `Pair` earlier in the notebook (`my_pair`).

Note that adding `my_pair` once again splits `stack.subset_stack` into 2 connected substacks.

In [9]:
stack.add_pairs([my_pair])
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')

len(stack.connected_substacks): 2


### View `stack.subset_stack`'s scene ID's 

When ordering data from ASF [HyP3](https://hyp3-docs.asf.alaska.edu/hyp3-docs/about/) or [HyP3+](https://hyp3-docs.asf.alaska.edu/hyp3-docs/about/hyp3_plus/) On-Demand Processing, it is helpful to provide a `Stack` as a list of scene IDs.

The `Stack.get_scene_ids()` method offers this functionality.

In [10]:
possibly_disconnected_subset_stack_ids = stack.get_scene_ids(stack.subset_stack)
possibly_disconnected_subset_stack_ids

[('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C'),
 ('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220323T225119_20220323T225146_042455_051017_6160'),
 ('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252',
  'S1A_IW_SLC__1SDV_20220227T225119_20220227T

### View the largest connected substack in `stack.connected_substacks`

`stack.get_scene_ids()` takes the optional argument `stack_list`. 

If you do not provide a stack_list, the method defaults to using the largest connected substack in `stack.connected_substacks`.

In [11]:
largest_fully_connected_subset_stack_ids = stack.get_scene_ids()
largest_fully_connected_subset_stack_ids


[('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C'),
 ('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E'),
 ('S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E',
  'S1A_IW_SLC__1SDV_20220323T225119_20220323T225146_042455_051017_6160'),
 ('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252',
  'S1A_IW_SLC__1SDV_20220227T225119_20220227T

In [12]:
len(largest_fully_connected_subset_stack_ids)

12

### You can view any stack_list contained in a `Stack` object, including the complete `stack.full_stack`, `stack.remove_list`, the possibly disconnected `stack.subset_stack`, or any of the stack_lists in `stack.connected_substacks`

In [13]:
remove_list_scene_ids = stack.get_scene_ids(stack.remove_list)
remove_list_scene_ids

[('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252'),
 ('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E'),
 ('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E'),
 ('S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53',
  'S1A_IW_SLC__1SDV_20220323T225119_20220323T225146_042455_051017_6160'),
 ('S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C',
  'S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E'),
 ('S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C',
  'S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252'),
 ('S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C',
  'S1A_IW_SLC__1SDV_20220227T225119_20220227T